In [34]:
import matplotlib as plt
print('matplotlib: {}'.format(plt.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn as sk
print('sklearn: {}'.format(sk.__version__))
from pandas_profiling import ProfileReport
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

matplotlib: 3.3.2
pandas: 1.1.3
sklearn: 0.23.2
seaborn: 0.11.0
numpy: 1.19.2


In [2]:
X_train = pd.read_csv("X_train.csv")

In [3]:
X_val=pd.read_csv("X_val.csv")
X_test = pd.read_csv("X_test.csv")

In [4]:
y_train = pd.read_csv("y_train.csv")
y_val = pd.read_csv("y_val.csv")
y_test = pd.read_csv("y_test.csv")

In [5]:
id_test = pd.read_csv("id_test.csv")

In [15]:
X_train = X_train.drop(columns=["Unnamed: 0"])

In [17]:
X_val = X_val.drop(columns=["Unnamed: 0"])

In [19]:
X_test = X_test.drop(columns=["Unnamed: 0"])

In [21]:
y_train =y_train.drop(columns=["Unnamed: 0"])

In [23]:
y_val = y_val.drop(columns=["Unnamed: 0"])

In [25]:
y_test = y_test.drop(columns=["Unnamed: 0"])

In [27]:
id_test = id_test.drop(columns=["Unnamed: 0"])

In [36]:
y_test.sample()

,SalePrice_Log
120,12.100712


In [35]:
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', linear_model.Ridge()))
models.append(('SVR', svm.SVR()))
models.append(('Lasso',linear_model.Lasso()))
models.append(('ElasticNet',ElasticNet()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold,scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -0.135828 (0.008469)
Ridge: -0.134779 (0.007722)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SVR: -0.336435 (0.017211)
Lasso: -0.302402 (0.020140)
ElasticNet: -0.299317 (0.020669)


In [37]:
# The best model is the ridge model

In [39]:
# Let's make a baseline

In [69]:
model = linear_model.Ridge()
model.fit(X_train, y_train)
predictions = model.predict(X_val)

In [42]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_val, predictions,squared=False)

0.14740600048182884

In [38]:
# Let's tune the hyperparameters

In [44]:
from sklearn.model_selection import GridSearchCV
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
model = linear_model.Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), scoring='neg_root_mean_squared_error')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_estimator_.alpha)


-0.13415896351556014
1.0


In [ ]:
# During EDA i took the log of the sale price to have a normally distributed feature
# Now I have to undo it to submit my results.

In [47]:
id_test

,Id
0,1461
1,1462
2,1463
3,1464
4,1465
...,...
1454,2915
1455,2916
1456,2917
1457,2918


In [67]:
# Final prediction

In [70]:
predictions2 = model.predict(X_test)

In [71]:
predictions2

array([[12.22054978],
       [12.09791646],
       [12.24402835],
       ...,
       [12.40451678],
       [11.7860125 ],
       [11.78241615]])

In [75]:
predictions2 = predictions2.flatten()

array([12.22054978, 12.09791646, 12.24402835, ..., 12.40451678,
       11.7860125 , 11.78241615])

In [73]:
mean_squared_error(y_test, predictions2,squared=False)

0.13114740678229392

In [48]:
SalePrice =  np.exp(predictions)

In [52]:
id_test.dtypes

Id    int64
dtype: object

In [65]:
(pd.DataFrame(data=[id_test, SalePrice]))

C:\Users\dalil\anaconda3\lib\site-packages\pandas\core\internals\construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


,0
0,Id 0 1461 1 1462 2 1463 3 ...
1,"[[180444.36608399986], [175805.545461731], [14..."


In [64]:
SalePrice

array([[180444.366084  ],
       [175805.54546173],
       [149924.44997892],
       [177897.72172331],
       [134730.74136049],
       [152510.44124616],
       [209442.41306561],
       [188087.76503118],
       [218375.09962404],
       [105005.12490608],
       [183187.52556638],
       [331816.55760791],
       [117131.79795728],
       [ 91454.62101548],
       [136949.8303605 ],
       [147909.41997556],
       [173458.244402  ],
       [138560.79150361],
       [205403.39314896],
       [169448.45861987],
       [438360.63591182],
       [253038.20316502],
       [172140.66058954],
       [121427.93263516],
       [115586.07567773],
       [161794.64474904],
       [124170.781475  ],
       [186087.54786834],
       [170863.98590591],
       [129169.56365554],
       [122534.52620021],
       [116176.34713586],
       [ 97357.9845925 ],
       [ 72911.75024374],
       [131872.57821373],
       [136888.6032407 ],
       [110425.29496089],
       [144254.14354723],
       [2665